# Projeto 2 - Classificador Automático de Sentimento 

### Bruno Costa e Vinicius Hsu

Você foi contratado por uma empresa parar analisar como os clientes estão reagindo a um determinado produto no Twitter. A empresa deseja que você crie um programa que irá analisar as mensagens disponíveis e classificará como "relevante" ou "irrelevante". Com isso ela deseja que mensagens negativas, que denigrem o nome do produto, ou que mereçam destaque, disparem um foco de atenção da área de marketing.<br /><br />
Como aluno de Ciência dos Dados, você lembrou do Teorema de Bayes, mais especificamente do Classificador Naive-Bayes, que é largamente utilizado em filtros anti-spam de e-mails. O classificador permite calcular qual a probabilidade de uma mensagem ser relevante dadas as palavras em seu conteúdo.<br /><br />
Para realizar o MVP (*minimum viable product*) do projeto, você precisa implementar uma versão do classificador que "aprende" o que é relevante com uma base de treinamento e compara a performance dos resultados com uma base de testes.<br /><br />
Após validado, o seu protótipo poderá também capturar e classificar automaticamente as mensagens da plataforma.

## Informações do Projeto

Prazo: 13/Set até às 23:59.<br />
Grupo: 1 ou 2 pessoas.<br /><br />
Entregáveis via GitHub: 
* Arquivo notebook com o código do classificador, seguindo as orientações abaixo.
* Arquivo Excel com as bases de treinamento e teste totalmente classificado.

**NÃO disponibilizar o arquivo com os *access keys/tokens* do Twitter.**


### Check 3: 

Até o dia 06 de Setembro às 23:59, o notebook e o xlsx devem estar no Github com as seguintes evidências: 
    * Conta no twitter criada.
    * Produto escolhido.
    * Arquivo Excel contendo a base de treinamento e teste já classificado.
    

Sugestão de leitura:<br />
http://docs.tweepy.org/en/v3.5.0/index.html<br />
https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/

___

## Preparando o ambiente

Instalando a biblioteca *tweepy* para realizar a conexão com o Twitter:

In [1]:
%%capture

#Instalando o tweepy
!pip install tweepy

Importando as Bibliotecas que serão utilizadas. Esteja livre para adicionar outras.

In [2]:
import tweepy
import math
import os.path
import pandas as pd
import json
import matplotlib.pyplot as plt
import os
from random import shuffle

___
## Autenticando no  Twitter

Para realizar a captura dos dados é necessário ter uma conta cadastrada no twitter:

* Conta: ***[Preencha aqui o id da sua conta. Ex: @fulano ]***


1. Caso ainda não tenha uma: https://twitter.com/signup
1. Depois é necessário registrar um app para usar a biblioteca: https://apps.twitter.com/
1. Dentro do registro do App, na aba Keys and Access Tokens, anotar os seguintes campos:
    1. Consumer Key (API Key)
    1. Consumer Secret (API Secret)
1. Mais abaixo, gere um Token e anote também:
    1. Access Token
    1. Access Token Secret
    
1. Preencha os valores no arquivo "auth.pass"

**ATENÇÃO**: Nunca divulgue os dados desse arquivo online (GitHub, etc). Ele contém as chaves necessárias para realizar as operações no twitter de forma automática e portanto é equivalente a ser "hackeado". De posse desses dados, pessoas mal intencionadas podem fazer todas as operações manuais (tweetar, seguir, bloquear/desbloquear, listar os seguidores, etc). Para efeito do projeto, esse arquivo não precisa ser entregue!!!

In [1]:
#Dados de autenticação do twitter:

#Coloque aqui o identificador da conta no twitter: @brunex887

#leitura do arquivo no formato JSON
with open('auth.pass') as fp:    
    data = json.load(fp)

#Configurando a biblioteca. Não modificar
auth = tweepy.OAuthHandler(data['consumer_key'], data['consumer_secret'])
auth.set_access_token(data['access_token'], data['access_token_secret'])

"#Dados de autenticação do twitter:\n\n#Coloque aqui o identificador da conta no twitter: @brunex887\n\n#leitura do arquivo no formato JSON\nwith open('auth.pass') as fp:    \n    data = json.load(fp)\n\n#Configurando a biblioteca. Não modificar\nauth = tweepy.OAuthHandler(data['consumer_key'], data['consumer_secret'])\nauth.set_access_token(data['access_token'], data['access_token_secret'])"

___
## Coletando Dados

Agora vamos coletar os dados. Tenha em mente que dependendo do produto escolhido, não haverá uma quantidade significativa de mensagens, ou ainda poder haver muitos retweets.<br /><br /> 
Configurando:

In [5]:
#Produto escolhido:
produto = 'askov'

#Quantidade mínima de mensagens capturadas:
n = 500
#Quantidade mínima de mensagens para a base de treinamento:
t = 300

#Filtro de língua, escolha uma na tabela ISO 639-1.
lang = 'pt'

"#Produto escolhido:\nproduto = 'askov'\n\n#Quantidade mínima de mensagens capturadas:\nn = 500\n#Quantidade mínima de mensagens para a base de treinamento:\nt = 300\n\n#Filtro de língua, escolha uma na tabela ISO 639-1.\nlang = 'pt'\n"

Capturando os dados do twitter:

In [6]:
#Cria um objeto para a captura
api = tweepy.API(auth)

#Inicia a captura, para mais detalhes: ver a documentação do tweepy
i = 1
msgs = []
for msg in tweepy.Cursor(api.search, q=produto, lang=lang).items():    
    msgs.append(msg.text.lower())
    i += 1
    if i > n:
        break

#Embaralhando as mensagens para reduzir um possível viés
shuffle(msgs)

'\n#Cria um objeto para a captura\napi = tweepy.API(auth)\n\n#Inicia a captura, para mais detalhes: ver a documentação do tweepy\ni = 1\nmsgs = []\nfor msg in tweepy.Cursor(api.search, q=produto, lang=lang).items():    \n    msgs.append(msg.text.lower())\n    i += 1\n    if i > n:\n        break\n\n#Embaralhando as mensagens para reduzir um possível viés\nshuffle(msgs)\n'

Salvando os dados em uma planilha Excel:

In [7]:
#Verifica se o arquivo não existe para não substituir um conjunto pronto
if not os.path.isfile('./{0}.xlsx'.format(produto)):
    
    #Abre o arquivo para escrita
    writer = pd.ExcelWriter('{0}.xlsx'.format(produto))

    #divide o conjunto de mensagens em duas planilhas
    dft = pd.DataFrame({'Treinamento' : pd.Series(msgs[:t])})
    dft.to_excel(excel_writer = writer, sheet_name = 'Treinamento', index = False)

    dfc = pd.DataFrame({'Teste' : pd.Series(msgs[t:])})
    dfc.to_excel(excel_writer = writer, sheet_name = 'Teste', index = False)

    #fecha o arquivo
    writer.save()

"\n#Verifica se o arquivo não existe para não substituir um conjunto pronto\nif not os.path.isfile('./{0}.xlsx'.format(produto)):\n    \n    #Abre o arquivo para escrita\n    writer = pd.ExcelWriter('{0}.xlsx'.format(produto))\n\n    #divide o conjunto de mensagens em duas planilhas\n    dft = pd.DataFrame({'Treinamento' : pd.Series(msgs[:t])})\n    dft.to_excel(excel_writer = writer, sheet_name = 'Treinamento', index = False)\n\n    dfc = pd.DataFrame({'Teste' : pd.Series(msgs[t:])})\n    dfc.to_excel(excel_writer = writer, sheet_name = 'Teste', index = False)\n\n    #fecha o arquivo\n    writer.save()\n"

___
## Classificando as Mensagens

Agora você deve abrir o arquivo Excel com as mensagens capturadas e classificar na Coluna B se a mensagem é relevante ou não.<br /> 
Não se esqueça de colocar um nome para a coluna na célula **B1**.<br /><br />
Fazer o mesmo na planilha de Controle.

___
## Montando o Classificador Naive-Bayes

Com a base de treinamento montada, comece a desenvolver o classificador. Escreva o seu código abaixo:

Opcionalmente: 
* Limpar as mensagens removendo os caracteres: enter, :, ", ', (, ), etc. Não remover emojis.<br />
* Corrigir separação de espaços entre palavras e/ou emojis.
* Propor outras limpezas/transformações que não afetem a qualidade da informação.



In [8]:
dados = pd.read_excel('askov.xlsx')
pd.DataFrame(dados)

,Treinamento,Relevância
0,pensando na garrafa de askov cheinha que eu qu...,Relevante
1,"beber cerveja, blue pinga, askov, vodka, cacha...",Irrelevante
2,@mjussani @_carolllm boa noite senhor dono do ...,Relevante
3,askov e corote de sabor são as pragas modernas,Irrelevante
4,@muller_daiana @camillemariam é um plano marav...,Irrelevante
5,@shasezin @datpqp como faz pra entrar na askov?,Irrelevante
6,"eu até tento beber askov, mas n desce por nd",Irrelevante
7,@ksoftjin queria ter comprado ma sminha amiga ...,Irrelevante
8,"crlh, deixei a askov c o cacau e saí fora...",Irrelevante
9,meu quarto tá cheirando askov de frutas vermelhas,Relevante


In [9]:
lista_splits=[]
for i in range(1,300):
    lista_splits.append(dados.iloc[i,0].split())

set_palavras=[] #Lista de Palavras Únicas
for i in lista_splits:
    for c in i:
        if c not in set_palavras:
            set_palavras.append(c)

## Relevantes

In [10]:
lista_relevantes=[]
for i in range(1,300):
    if dados.Relevância[i] == "Relevante":
        lista_relevantes.append(dados.Treinamento[i].split())
        
todas_relevantes=[] #Lista de Palavras Únicas

for i in lista_relevantes:
    for c in i:
        todas_relevantes.append(c)
        
dic_relev={}

for i in set_palavras:
    dic_relev[i]=((todas_relevantes.count(i)+1)/(len(todas_relevantes)+3879))


## Irrelevantes

In [11]:
lista_irrelevantes=[]

for i in range(1,300):
    if dados.Relevância[i] == "Irrelevante":
        lista_irrelevantes.append(dados.Treinamento[i].split())
        
todas_irrelevantes=[] #Lista de Palavras Únicas

for i in lista_irrelevantes:
    for c in i:
        todas_irrelevantes.append(c)
        
dic_irrelev={}

for i in set_palavras:
    dic_irrelev[i]=((todas_irrelevantes.count(i)+1)/(len(todas_irrelevantes)+3879))

## Re-Avaliando os Tweets

In [12]:
dados = pd.read_excel('askov.xlsx')
dadosteste = pd.read_excel('askov.xlsx', sheetname="Teste")
dadosteste["Performance"] = "Relevante"

lista_splits2=[]
for i in range(1,200):
    lista_splits2.append(dadosteste.iloc[i,0].split())

for ind,i in enumerate(lista_splits2):
    chance_relev=1
    chance_irrelev=1
    
    for c in i:
        if c in dic_relev:
            chance_relev = chance_relev*dic_relev[c]
        if c in dic_irrelev:    
            chance_irrelev = chance_irrelev*dic_irrelev[c]
        else:
            chance_relev = chance_relev*1
            chance_irrelev = chance_irrelev*1
    if chance_relev>chance_irrelev:
        dadosteste.Performance[ind] = "Relevante"
        
    if chance_irrelev>chance_relev:
        dadosteste.Performance[ind] = "Irrelevante"
        
pd.DataFrame(dadosteste)

,Teste,Relevância,Performance
0,"@oisabrinex kkkkkkkkkk não, tem de catuaba, as...",Irrelevante,Irrelevante
1,"ontem eu bebi blue pinga, askov, tequila, dreh...",Irrelevante,Irrelevante
2,@_mizonnes mano\nvcs colocaram alcool no copo ...,Irrelevante,Irrelevante
3,@alice08menezes com pt de askov nem tanto,Irrelevante,Irrelevante
4,rt @mellocollela: segue lista de álcool pro fe...,Irrelevante,Relevante
5,quero beber askov de maracujaaa,Relevante,Irrelevante
6,"@vakeno nem cerveja e nem askov pffff, vamos d...",Irrelevante,Irrelevante
7,eu acabei o copo de askov em 5 minutos mas to ...,Irrelevante,Irrelevante
8,ptzao no rolê é oq? cheiro do guaraná skin com...,Irrelevante,Irrelevante
9,saudades da minha boquinha rosinha de askov,Relevante,Relevante


___
## Verificando a performance

Agora você deve testar o seu Classificador com a base de Testes.<br /><br /> 

Você deve extrair as seguintes medidas:
* Porcentagem de positivos falsos (marcados como relevante mas não são relevantes)
* Porcentagem de positivos verdadeiros (marcado como relevante e são relevantes)
* Porcentagem de negativos verdadeiros (marcado como não relevante e não são relevantes)
* Porcentagem de negativos falsos (marcado como não relevante e são relevantes)

Opcionalmente:
* Criar categorias intermediárias de relevância baseado na diferença de probabilidades. Exemplo: muito relevante, relevante, neutro, irrelevante e muito irrelevante.

In [52]:
positivos_falsos = dadosteste[(dadosteste.Relevância== "Irrelevante") & (dadosteste.Performance== "Relevante")]
positivos_verdadeiros = dadosteste[(dadosteste.Relevância== "Relevante") & (dadosteste.Performance== "Relevante")]
negativos_verdadeiros = dadosteste[(dadosteste.Relevância== "Irrelevante") & (dadosteste.Performance== "Irrelevante")]
negativos_falsos = dadosteste[(dadosteste.Relevância== "Relevante") & (dadosteste.Performance== "Irrelevante")]

porc_positivos_falsos = positivos_falsos.shape[0]*100/dadosteste.shape[0]
porc_positivos_verdadeiros = positivos_verdadeiros.shape[0]*100/dadosteste.shape[0]
porc_negativos_verdadeiros = negativos_verdadeiros.shape[0]*100/dadosteste.shape[0]
porc_negativos_falsos = negativos_falsos.shape[0]*100/dadosteste.shape[0]

print("Porcentagem de positivos falsos: {}%" .format(porc_positivos_falsos))
print("Porcentagem de positivos verdadeiros: {}%" .format(porc_positivos_verdadeiros))
print("Porcentagem de negativos verdadeiros: {}%" .format(porc_negativos_verdadeiros))
print("Porcentagem de negativos falsos: {}%" .format(porc_negativos_falsos))

Porcentagem de positivos falsos: 11.5%
Porcentagem de positivos verdadeiros: 3.0%
Porcentagem de negativos verdadeiros: 59.5%
Porcentagem de negativos falsos: 26.0%


In [27]:
pd.crosstab(dadosteste.Relevância, dadosteste.Performance, normalize=True)*100

Performance,Irrelevante,Relevante
Relevância,,
Irrelevante,59.5,11.5
Relevante,26.0,3.0


___
## Concluindo

Escreva aqui a sua conclusão.<br /> 
Faça um comparativo qualitativo sobre as medidas obtidas.<br />
Explique como são tratadas as mensagens com dupla negação e sarcasmo.<br />
Proponha um plano de expansão. Por que eles devem continuar financiando o seu projeto?<br />

Opcionalmente: 
* Discorrer por que não posso alimentar minha base de Treinamento automaticamente usando o próprio classificador, aplicado a novos tweets.
* Propor diferentes cenários de uso para o classificador Naive-Bayes. Cenários sem intersecção com este projeto.
* Sugerir e explicar melhorias reais no classificador com indicações concretas de como implementar (não é preciso codificar, mas indicar como fazer e material de pesquisa sobre o assunto).


# Resultados:


Os resultados obtidos pelo classificador montado com base em tweets sobre a bebida askov foram de certa forma satisfatórios. Obtivemos um acerto de 62.5%, do total, que se levado em conta nossa estimativa inicial, uma vez que é um classificador simples, nos surpreendeu com um resultado muito a cima do esperado. O ponto fraco do nosso classificador foi a classificação de tweets relevante, que apesar de poucos - e por isso (falta de dados para estabelecer uma base sólida), talvez um resultado tão fraco - obteve apenas um acerto de aproximadamente 10% dos tweets positivos, enquanto os negativos tiveram um acerto de cerca de 70%. 

As mensagens com dupla negação ou sarcasmo não foram bem interpretadas pelo classificador, sendo classificadas contrariamente em relevância ao que deveriam ser classificadas. O classificador não consegue identificar sarcasmo nos tweets ja que analisa palavra por palavra e não as frases como um todo, ou seja tweets duplamente negativos ou com sarcasmo não foram previstos na montagem do classificador e por isso apresentam valores que não condizem com a realidade, mas que foram previstos pela expectativa da dupla.

Com a continuação do financiamento do projeto é possível aprimorar o classificador obtendo um maior número de tweets a serem trabalhados como uma base de dados. O resultado obtido nessa primeira realização foi de certa forma satisfatório e com certeza a cima do que era experado para uma primeira iteração do classificador. Por isso, com a continuação do financiamento para aprimoramento do objeto de estudo, uma melhora significativa deverá ser observada aproximando-se do ideal traçado pela dupla.